In [1]:
import numpy as np
import pandas as pd

pos = pd.read_excel('data/pos.xls', header=None)
pos['mark'] = 1
neg = pd.read_excel('data/neg.xls', header=None)
neg['mark'] = 0

pn = pd.concat([pos,neg],ignore_index=True)

In [2]:
maxlen = 200
# 最大句长，超过最大句长进行截断
min_count = 20
# 最小字频，低于最小字频舍弃

content = ''.join(pn[0])
dict = pd.Series(list(content)).value_counts()
dict = dict[dict > min_count]
dict[:] = list(range(len(dict)))
word_set = set(dict.index)
# print(dict)
# print(word_set)

In [3]:
def doc2num(sen, maxlen):
    # 将字对应到id
    sen = [i for i in sen if i in word_set]
    sen = sen[:maxlen]
    return list(dict[sen])

pn['doc2num'] = pn[0].apply(lambda s: doc2num(s, maxlen))
# print(pn['doc2num'])

In [4]:
idx = list(range(len(pn)))
np.random.shuffle(idx)
pn = pn.loc[idx]
# 通过loc值取数据

x = np.array(list(pn['doc2num']))
y = np.array(list(pn['mark']))
y = y.reshape((-1,1)) #调整标签形状

In [5]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
import sys
sys.setrecursionlimit(10000)

In [6]:
#建立模型
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen,len(dict)))) 
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#单个one hot矩阵的大小是maxlen*len(abc)的，非常消耗内存
#为了方便低内存的PC进行测试，这里使用了生成器的方式来生成one hot矩阵
#仅在调用时才生成one hot矩阵
#可以通过减少batch_size来降低内存使用，但会相应地增加一定的训练时间
batch_size = 128
train_num = 15000

#不足则补全0行
gen_matrix = lambda z: np.vstack((np_utils.to_categorical(z, len(dict)), np.zeros((maxlen-len(z), len(dict)))))

def data_generator(data, labels, batch_size): 
    batches = [list(range(batch_size*i, min(len(data), batch_size*(i+1)))) for i in range(len(data)/batch_size+1)]
    while True:
        for i in batches:
            xx = np.zeros((maxlen, len(dict)))
            xx, yy = np.array(map(gen_matrix, data[i])), labels[i]
            yield (xx, yy)

model.fit_generator(data_generator(x[:train_num], y[:train_num], batch_size), samples_per_epoch=train_num, nb_epoch=30)

model.evaluate_generator(data_generator(x[train_num:], y[train_num:], batch_size), val_samples=len(x[train_num:]))

def predict_one(s): #单个句子的预测函数
    s = gen_matrix(doc2num(s, maxlen))
    s = s.reshape((1, s.shape[0], s.shape[1]))
    return model.predict_classes(s, verbose=0)[0][0]

Instructions for updating:
Please use Model.fit, which supports generators.


TypeError: fit_generator() got an unexpected keyword argument 'samples_per_epoch'